In [34]:
import os
import pandas as pd
from modules.util import numerical_to_float


df1 = pd.read_csv('data/data_files/BG_meteo_30min_20230101_20230801.csv', sep=',', na_values=['NaN']).drop(0)
df2 = pd.read_csv('data/data_files/GW_meteo_30min_20230101_20230801.csv', sep=',', na_values=['NaN']).drop(0)
df3 = pd.read_csv('data/data_files/BG_meteo_30min_20240401_20240608.csv', sep=';', na_values=['NaN']).drop(0)
df4 = pd.read_csv('data/data_files/GW_meteo_30min_20240401_20240608.csv', sep=';', na_values=['NaN']).drop(0)

df1 = df1.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df2 = df2.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df3 = df3.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df4 = df4.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)

In [35]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)

Index(['TIMESTAMP_START', 'kurzwEinstrahlung_300cm', 'kurzwAusstrahlung_300cm',
       'Nettostrahlung_300cm', 'diffuseEinstrahlung_160cm',
       'PPFDEinstrahlung_300cm', 'PPFDAusstrahlung_300cm', 'Bodenwaermefluss',
       'Luftdruck', 'Lufttemperatur_40cm', 'Lufttemperatur_200cm',
       'RelativeFeuchte_40cm', 'RelativeFeuchte_200cm',
       'Wasserdampfdruck_200cm', 'Wasserdampfdefizit_200cm',
       'Niederschlag_I_155cm', 'Niederschlag_II_150cm', 'Windgeschw_380cm',
       'Windrichtung_380cm', 'Bodentemp_30cm_1', 'Bodentemp_30cm_2',
       'Bodentemp_30cm_3', 'Bodenfeuchte_30cm_1', 'Bodenfeuchte_30cm_2',
       'Bodenfeuchte_30cm_3'],
      dtype='object')
Index(['TIMESTAMP_START', 'kurzwEinstrahlung_43m', 'kurzwAusstrahlun_43m',
       'langwEinstrahlung_43m', 'langwAusstrahlung_43m', 'Luftdruck_43m',
       'Lufttemperatur_43m', 'RelativeFeuchte_43m', 'Wasserdampfdruck_43m',
       'Wasserdampfdefizit_43m', 'Niederschlag_43m', 'Windgeschw_I_43m',
       'Windgeschw_II_43m', 

In [36]:
df1["kurzwEinstrahlung"] = df1["kurzwEinstrahlung_300cm"]
df2["kurzwEinstrahlung"] = df2["kurzwEinstrahlung_43m"]
df3["kurzwEinstrahlung"] = df3["kurzwEinstrahlung_300cm"]
df4["kurzwEinstrahlung"] = df4["kurzwEinstrahlung_43m"]

df1["kurzwAusstrahlung"] = df1["kurzwAusstrahlung_300cm"]
df2["kurzwAusstrahlung"] = df2["kurzwAusstrahlun_43m"]
df3["kurzwAusstrahlung"] = df3["kurzwAusstrahlung_300cm"]
df4["kurzwAusstrahlung"] = df4["kurzwAusstrahlung_43m"]

df1["Bodentemp_1_30cm"] = df1["Bodentemp_30cm_1"]

df2["Luftdruck"] = df2["Luftdruck_43m"]
df4["Luftdruck"] = df4["Luftdruck_43m"]

df1["Wasserdampfdefizit"] = df1["Wasserdampfdefizit_200cm"]
df2["Wasserdampfdefizit"] = df2["Wasserdampfdefizit_43m"]
df3["Wasserdampfdefizit"] = df3["Wasserdampfdefizit_200cm"]
df4["Wasserdampfdefizit"] = df4["Wasserdampfdefizit_43m"]

df1["Wasserdampfdruck"] = df1["Wasserdampfdruck_200cm"]
df2["Wasserdampfdruck"] = df2["Wasserdampfdruck_43m"]
df3["Wasserdampfdruck"] = df3["Wasserdampfdruck_200cm"]
df4["Wasserdampfdruck"] = df4["Wasserdampfdruck_43m"]

df1["RelativeFeuchte"] = df1["RelativeFeuchte_200cm"]
df2["RelativeFeuchte"] = df2["RelativeFeuchte_43m"]
df3["RelativeFeuchte"] = df3["RelativeFeuchte_200cm"]
df4["RelativeFeuchte"] = df4["RelativeFeuchte_2m"]

df1["Windgeschw"] = df1["Windgeschw_380cm"]
df2["Windgeschw"] = df2["Windgeschw_I_43m"]
df3["Windgeschw"] = df3["Windgeschw_380cm"]
df4["Windgeschw"] = df4["Windgeschw_I_43m"]


cols_to_keep = ["TIMESTAMP_START", "kurzwEinstrahlung", "kurzwAusstrahlung", "Bodentemp_1_30cm", "Luftdruck", "Wasserdampfdefizit", "RelativeFeuchte", "Windgeschw"]
cols_to_convert = ["kurzwEinstrahlung", "kurzwAusstrahlung", "Bodentemp_1_30cm", "Luftdruck", "Wasserdampfdefizit", "RelativeFeuchte", "Windgeschw"]

df1 = df1[cols_to_keep]
df2 = df2[cols_to_keep]
df3 = df3[cols_to_keep]
df4 = df4[cols_to_keep]

df1 = numerical_to_float(df1, cols_to_convert)
df2 = numerical_to_float(df2, cols_to_convert)
df3 = numerical_to_float(df3, cols_to_convert)
df4 = numerical_to_float(df4, cols_to_convert)


In [37]:
# merge 2023 and 2024 data, respectively
df1["Location"] = 0
df2["Location"] = 1
df3["Location"] = 0
df4["Location"] = 1


# concat both
df = pd.concat([df_23, df_24])

# merge some cols
df.head()

,TIMESTAMP_START,kurzwEinstrahlung_BG,kurzwAusstrahlung_BG,Bodentemp_1_30cm_BG,Luftdruck_BG,Wasserdampfdefizit_BG,RelativeFeuchte_BG,Windgeschw_BG,kurzwEinstrahlung_GW,kurzwAusstrahlung_GW,Bodentemp_1_30cm_GW,Luftdruck_GW,Wasserdampfdefizit_GW,RelativeFeuchte_GW,Windgeschw_GW
0,2023-01-01 00:00:00,-3.695000,-0.092667,8.993333,984.325000,6.826310,58.748000,2.980000,-1.324667,0.857667,NaN,956.0,6.556116,57.563333,8.014000
1,2023-01-01 00:30:00,-4.819000,-0.026667,9.003333,984.493333,6.667064,59.098000,3.113000,-1.767333,1.208333,NaN,956.0,6.466390,57.796667,7.907333
2,2023-01-01 01:00:00,-4.449333,0.007667,9.006667,984.536667,6.568903,59.369667,2.817000,-1.716333,1.199667,NaN,956.0,6.307071,58.376667,7.362667
3,2023-01-01 01:30:00,-3.713000,-0.130333,9.033333,984.500667,6.618655,59.135333,2.737667,-1.427000,1.012667,NaN,956.0,6.237201,58.576667,7.032000
4,2023-01-01 02:00:00,-4.892000,0.062000,9.046667,984.518667,6.407977,59.696333,2.594667,-1.813667,1.251667,NaN,956.0,6.144474,58.726667,7.547667


In [38]:
def nan_percentage(df):
    """
    Returns a DataFrame containing the percentage of NaN values in each column of the input DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    
    Returns:
    pd.DataFrame: A DataFrame with columns 'Column' and 'NaN_Percentage' showing the percentage of NaN values for each column.
    """
    # Calculate the percentage of NaN values for each column
    nan_percentage = df.isna().mean() * 100
    
    # Create a new DataFrame with the results
    nan_df = pd.DataFrame({
        'Column': nan_percentage.index,
        'NaN_Percentage': nan_percentage.values
    })
    
    return nan_df

In [39]:
df_nan = nan_percentage(df)
df_nan.to_csv("nans.csv")